# Old (Self-Consistency)

In [ ]:
!pip install nltk
import json
import jsonlines
import re
import json
from difflib import SequenceMatcher
import string
from difflib import SequenceMatcher
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

def ideas_less_to_json(_data):
    pattern = r"{\n(.*?\n})\n}"
    substring = re.search(pattern, _data, re.DOTALL).group(1)
    # if ': {' in substring:
    #     substring = substring.replace(': {', ':[')
    #     substring = substring.replace('}', ']')
    
    substring_json = json.loads('{'+substring+'}')
    # print(type(substring_json))
    return substring_json

with jsonlines.open('../teste-progresso/04-bilou/annotations-medical_specialist-dpoc-json.jsonl') as reader:
        annotated_dataset = [line for line in reader]
for doc in annotated_dataset:
    if doc['doc_id'] == '6c118cb7-47ad-41fa-b429-51c7232e7a66':
        specialist_annotations = doc['annotations']
n_specialist_annotations = len(specialist_annotations)
print('Number of ideas from specialist:',len(specialist_annotations))
ideas = []
for idea in specialist_annotations:
    ideas.append(idea[0])
print(ideas)  
print('========================')  

with open('ideas-one-few-6c118cb7-47ad-41fa-b429-51c7232e7a66.json', "r", encoding='utf8') as file:
    llama_annotated = json.load(file)


for i in range(len(llama_annotated['response'])):
    run = llama_annotated['response'][i] 
    llama_annotated['response'][i] = ideas_shot_to_json(run)
    print(llama_annotated['response'][i])
    idea_list = []
    for idea in llama_annotated['response'][i]['annotations'].values():
        if len(idea) > 1:
            idea_list = idea
        else:
            idea_list.append(idea[0])
    llama_annotated['response'][i]['grouped_ideas'] = idea_list
def get_top_ideas(response):
    dict_ideas = {}
    printed_lengths = set()
    length_count = {}
    max_length = 0
    for data in response['response']:
        for idea in data['grouped_ideas']:
            idea_no_accents = idea.replace('á', 'a').replace('ã', 'a').replace('é', 'e').replace('ê', 'e').replace('í', 'i').replace('ó', 'o').replace('ô', 'o').replace('õ', 'o').replace('ú', 'u').replace('ç', 'c')
            dict_ideas[idea_no_accents] = dict_ideas.get(idea_no_accents,0)+1
        length = len(data['grouped_ideas'])
        if length not in printed_lengths:
            printed_lengths.add(length)
        if length > max_length:
            max_length = length
        length_count[length] = length_count.get(length, 0) + 1
    print("Amount of objects with each length:")

    for length, count in length_count.items():
        print(f"Length {length}: {count} objects")

    # Print the most used values throughout all objects in descending order
    # value_count = {}
    # for key in dict_ideas.items():
    #     value_count[key] = value_count.get(value, 0) + 1
    sorted_values = sorted(dict_ideas.items(), key=lambda x: x[1], reverse=True)
    print("Most used values:")
    value_list = []
    for value, count in sorted_values:
        value_list.append(value)
        print(f"{value}: {count} times")
    # print(sorted_values[:max_length])
    return value_list[:max_length],max_length


llama_annotations, n_llama_annotations = get_top_ideas(llama_annotated)

# for llama_idea in llama_annotations:
#     for specialist_idea in specialist_annotations:
#         if llama_idea in specialist_idea:
#             print(f"{llama_idea} is contained in {specialist_idea}")
#         if specialist_idea in llama_idea:
#             print(f"{specialist_idea} is contained in {llama_idea}")
print('relatedness section','==============================================')
llama_dict_precision = {}
spec_i_blacklist = []
for llama_idea in llama_annotations:
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    llama_dict_precision[llama_idea] = {'precision':0,'specialist_idea_contained':[],'is_fully_contained':False}
    spec_i = 0

    for specialist_idea_info in specialist_annotations:
        if spec_i not in spec_i_blacklist:
            specialist_idea = specialist_idea_info[0]
            specialist_idea_no_accents = specialist_idea.replace('á', 'a').replace('ã', 'a').replace('é', 'e').replace('ê', 'e').replace('í', 'i').replace('ó', 'o').replace('ô', 'o').replace('õ', 'o').replace('ú', 'u').replace('ç', 'c')
            # Define the stopwords and punctuation
            stopwords = nltk.corpus.stopwords.words('portuguese')
            punctuation = string.punctuation

            # Remove stopwords and punctuation from the strings
            llama_idea_clean = ' '.join([word for word in llama_idea.split() if word.lower() not in stopwords and word not in punctuation])
            specialist_idea_clean = ' '.join([word for word in specialist_idea_no_accents.split() if word.lower() not in stopwords and word not in punctuation])
            llama_idea_clean_tokens = (llama_idea_clean).lower().split()
            specialist_idea_clean_tokens = specialist_idea_clean.lower().split()
            # print(llama_idea_clean_tokens)
            # print(specialist_idea_clean_tokens)

            l_sublist_of_s = set(llama_idea_clean_tokens).issubset(set(specialist_idea_clean_tokens))
            s_sub_of_l = set(specialist_idea_clean_tokens).issubset(set(llama_idea_clean_tokens))
            # Calculate the relatedness ratios
            matcher = SequenceMatcher(lambda x: x in [' ', '\n'] or x.lower() in stopwords or x in punctuation, llama_idea, specialist_idea)
            a_to_b_ratio = matcher.ratio()

            # Print the relatedness ratios
            # print('relatedness','========================')
            if s_sub_of_l and l_sublist_of_s:
                llama_dict_precision[llama_idea]['precision'] = 1
                llama_dict_precision[llama_idea]['specialist_idea_contained'].append(specialist_idea)
                llama_dict_precision[llama_idea]['is_fully_contained'] = True
                spec_i_blacklist.append(spec_i)   

                print(llama_idea_clean,'||||', specialist_idea_clean)
                print('100% match')
            elif s_sub_of_l:
                llama_dict_precision[llama_idea]['precision'] += a_to_b_ratio
                llama_dict_precision[llama_idea]['specialist_idea_contained'].append(specialist_idea)
                llama_dict_precision[llama_idea]['is_fully_contained'] = False
                spec_i_blacklist.append(spec_i)   

                print(llama_idea_clean,'||||', specialist_idea_clean)
                print(f'Complete match from specialist_idea to llama_idea: {s_sub_of_l}')
                print(f'Relatedness of llama_idea to specialist_idea: {a_to_b_ratio}')
            elif l_sublist_of_s:
                llama_dict_precision[llama_idea]['precision'] += a_to_b_ratio
                llama_dict_precision[llama_idea]['specialist_idea_contained'].append(specialist_idea)
                llama_dict_precision[llama_idea]['is_fully_contained'] = False
                spec_i_blacklist.append(spec_i)   

                print(llama_idea_clean,'||||', specialist_idea_clean)
                print(f'Complete match from llama_idea to specialist_idea: {l_sublist_of_s}')
                print(f'Relatedness of llama_idea to specialist_idea: {a_to_b_ratio}')
        # elif a_to_b_ratio > 0.17:
        spec_i += 1
        
    print(specialist_annotations)
    print(llama_dict_precision[llama_idea])
    print('################################################')



In [ ]:
def ideas_shot_to_json(_data):
    pattern = r"{\n(.*?)\n}"
    substring = re.search(pattern, _data, re.DOTALL).group(1)
    # if ': {' in substring:
    #     substring = substring.replace(': {', ':[')
    #     substring = substring.replace('}', ']')
    try:
        substring_json = eval('{'+substring+'}')
    except:
        substring = substring.replace("[['", "['")
        substring_json = eval('{'+substring+'}')
    # print(type(substring_json))
    return substring_json

with open('ideas-one-few-6c118cb7-47ad-41fa-b429-51c7232e7a66.json', "r", encoding='utf8') as file:
    llama_annotated = json.load(file)


for i in range(len(llama_annotated['response'])):
    run = llama_annotated['response'][i] 
    llama_annotated['response'][i] = ideas_shot_to_json(run)

    idea_list = []
    for idea in llama_annotated['response'][i]['annotations']:
        if len(idea) > 1:
            idea_list = idea
        else:
            idea_list.append(idea[0])
    llama_annotated['response'][i]['grouped_ideas'] = idea_list

def get_top_ideas_w_cat(response):
    print('====!!!======!!!')
    print(response['response'][1]['annotations'])
    dict_ideas = {}
    dict_ideas_w_category = {}
    printed_lengths = set()
    length_count = {}
    max_length = 0
    for data in response['response']:
        for idea_block in data['annotations']:
            idea = idea_block[0]
            category = []
            for cat in idea_block[1]:
                blacklist = ['certo','errado','simples','encapsulado','jargão']
                if cat not in blacklist:
                    category.append(cat)
            
            # print('category',category)
            # print('for annotations,',idea_block)
            # print(idea)
            idea_no_accents = idea.replace('á', 'a').replace('ã', 'a').replace('é', 'e').replace('ê', 'e').replace('í', 'i').replace('ó', 'o').replace('ô', 'o').replace('õ', 'o').replace('ú', 'u').replace('ç', 'c')
            dict_ideas[idea_no_accents] = dict_ideas.get(idea_no_accents,0)+1
            # if dict_ideas_w_category.get(idea,0) == 0:
            #     dict_ideas_w_category[idea] = {str(category): dict_ideas_w_category[idea].get(str(category),0)+1}
            # elif dict_ideas_w_category[idea] != category:
            #     dict_ideas_w_category[idea] = {str(category): dict_ideas_w_category[idea].get(str(category),0)+1}
            #     print('======= CATEGORIA DIVERGENTE  ========================')
            #     print('old',dict_ideas_w_category[idea],'new',category)
            if dict_ideas_w_category.get(idea_no_accents,0) == 0:
                dict_ideas_w_category[idea_no_accents] = {}
            dict_ideas_w_category[idea_no_accents][str(category)] = dict_ideas_w_category[idea_no_accents].get(str(category),0)+1
            print('dict_ideas_w_category',dict_ideas_w_category[idea_no_accents])

        length = len(data['annotations'])
        if length not in printed_lengths:
            printed_lengths.add(length)
        if length > max_length:
            max_length = length
        length_count[length] = length_count.get(length, 0) + 1
    # for idea in dict_ideas_w_category:
    #     max_category = max(dict_ideas_w_category[idea], key=dict_ideas_w_category[idea].get)
    #     dict_ideas_w_category[idea] = max_category
    print("Amount of objects with each length:")

    for length, count in length_count.items():
        print(f"Length {length}: {count} objects")

    # Print the most used values throughout all objects in descending order
    # value_count = {}
    # for key in dict_ideas.items():
    #     value_count[key] = value_count.get(value, 0) + 1
    sorted_values = sorted(dict_ideas.items(), key=lambda x: x[1], reverse=True)
    print("Most used values:")
    value_list = []
    for value, count in sorted_values:
        value_list.append(value)
        print(f"{value} {dict_ideas_w_category[value]}: {count} times")
    # print(sorted_values[:max_length])
    return value_list[:max_length],max_length,dict_ideas_w_category


llama_annotations, n_llama_annotations,annotations_w_category = get_top_ideas_w_cat(llama_annotated)

# for llama_idea in llama_annotations:
#     for specialist_idea in specialist_annotations:
#         if llama_idea in specialist_idea:
#             print(f"{llama_idea} is contained in {specialist_idea}")
#         if specialist_idea in llama_idea:
#             print(f"{specialist_idea} is contained in {llama_idea}")
print('relatedness section','==============================================')
llama_dict_precision = {}
spec_i_blacklist = []
for llama_idea in llama_annotations:
    print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    llama_dict_precision[llama_idea] = {'precision':0,'specialist_idea_contained':[],'is_fully_contained':False}
    spec_i = 0

    for specialist_idea_info in specialist_annotations:
        if spec_i not in spec_i_blacklist:
            specialist_idea = specialist_idea_info[0]
            specialist_idea_no_accents = specialist_idea.replace('á', 'a').replace('ã', 'a').replace('é', 'e').replace('ê', 'e').replace('í', 'i').replace('ó', 'o').replace('ô', 'o').replace('õ', 'o').replace('ú', 'u').replace('ç', 'c')
            # Define the stopwords and punctuation
            stopwords = nltk.corpus.stopwords.words('portuguese')
            punctuation = string.punctuation

            # Remove stopwords and punctuation from the strings
            llama_idea_clean = ' '.join([word for word in llama_idea.split() if word.lower() not in stopwords and word not in punctuation])
            specialist_idea_clean = ' '.join([word for word in specialist_idea_no_accents.split() if word.lower() not in stopwords and word not in punctuation])
            llama_idea_clean_tokens = (llama_idea_clean).lower().split()
            specialist_idea_clean_tokens = specialist_idea_clean.lower().split()
            # print(llama_idea_clean_tokens)
            # print(specialist_idea_clean_tokens)

            l_sublist_of_s = set(llama_idea_clean_tokens).issubset(set(specialist_idea_clean_tokens))
            s_sub_of_l = set(specialist_idea_clean_tokens).issubset(set(llama_idea_clean_tokens))
            # Calculate the relatedness ratios
            matcher = SequenceMatcher(lambda x: x in [' ', '\n'] or x.lower() in stopwords or x in punctuation, llama_idea, specialist_idea)
            a_to_b_ratio = matcher.ratio()

            # Print the relatedness ratios
            # print('relatedness','========================')
            if s_sub_of_l and l_sublist_of_s:
                llama_dict_precision[llama_idea]['precision'] = 1
                llama_dict_precision[llama_idea]['specialist_idea_contained'].append(specialist_idea)
                llama_dict_precision[llama_idea]['is_fully_contained'] = True
                spec_i_blacklist.append(spec_i)   

                print(llama_idea_clean,'||||', specialist_idea_clean)
                print(annotations_w_category[llama_idea])
                print('100% match')
            elif s_sub_of_l:
                llama_dict_precision[llama_idea]['precision'] += a_to_b_ratio
                llama_dict_precision[llama_idea]['specialist_idea_contained'].append(specialist_idea)
                llama_dict_precision[llama_idea]['is_fully_contained'] = False
                spec_i_blacklist.append(spec_i)   

                print(llama_idea_clean,'||||', specialist_idea_clean)
                print(annotations_w_category[llama_idea])
                print(f'Complete match from specialist_idea to llama_idea: {s_sub_of_l}')
                print(f'Relatedness of llama_idea to specialist_idea: {a_to_b_ratio}')
            elif l_sublist_of_s:
                llama_dict_precision[llama_idea]['precision'] += a_to_b_ratio
                llama_dict_precision[llama_idea]['specialist_idea_contained'].append(specialist_idea)
                llama_dict_precision[llama_idea]['is_fully_contained'] = False
                spec_i_blacklist.append(spec_i)   

                print(llama_idea_clean,'||||', specialist_idea_clean)
                print(annotations_w_category[llama_idea])
                print(f'Complete match from llama_idea to specialist_idea: {l_sublist_of_s}')
                print(f'Relatedness of llama_idea to specialist_idea: {a_to_b_ratio}')
        # elif a_to_b_ratio > 0.17:
        spec_i += 1
        
    print(specialist_annotations)
    print(llama_dict_precision[llama_idea])
    print('################################################')

In [ ]:
import pandas as pd
import json
import re
import os

df = pd.read_csv('../teste-progresso/02-anotacoes/annotations-medical_specialist-dpoc.csv')
df.head()
annotation_id = '05a4a8fe-6c7b-400e-98b8-e46521e6ddd8'
filtered_df = df[df['annotation id'] == annotation_id]
filtered_df
organization_level = filtered_df['organization level'].values[0]
organization_level
with open('organization-less-05a4a8fe-6c7b-400e-98b8-e46521e6ddd8.json', "r", encoding='utf8') as file:
    llama_annotated = json.load(file)

votes = {}
for org in llama_annotated['response']:
    org_string = org
    number = re.findall(r'\d+', org_string)
    if number:
        number = int(number[0])
        votes[number] = votes.get(number, 0) + 1
if len(list(votes.keys())) == 1:
    print('different choice of votes')
else:
    # return list(votes.keys())[0]
    max_key = max(votes, key=lambda k: votes[k])
    print(max_key)
    print(list(votes.keys())[0])


file_names = os.listdir('/')
file_names = [file for file in os.listdir('/') if file.endswith('.json')]


# Comparison of the annotations (TF-IDF and arbitrary examples)